#Final Project 261 
### Team 9 - Ashwini, Rochelle, Mackenzie, Simran, Harinandan

### Summer 2021, section [w261_su21_section2]

d 

Flight delays are frequent all over the world. As per research overall 20% of airline flights arrive more that 15 mitues late which costs several teams of billion dollers annually . This scenario makes the prediction of flight delays a primary issue for airlines and travelers. The main goal of this project is to implement a predictor of the departure delay of a scheduled flight due to weather condition .

   For this project we are given with flight information , weather forecast and station datasets.  The given dataset were too large  and complex to be handled by traditional data analysis solutions . Therefore data has been analyzed and mined using parallel algorithms implemented using spark and mapreduced , executed on a cloud platform using Databrick and Azure could storage . 
   
   ####Dataset :
   
   <img src="https://github.com/AshwiniBhingare203/images261/blob/main/dataset.PNG?raw=true" width="30%">

## Table of Contents



* [Section 1 - Question Formulation](#Section1)
* [Section 2 - Algorithm Explanation](#Section2)
* [Section 3 - EDA & Challenges](#Section3)
* [Section 4 - Algorithm Implementation](#Section4)
* [Section 5 - Conclusion](#Section5)
* [Section 6 - Course Concepts](#Section6)

## Understanding the problem  <a name="Section1"></a>

### Why is the problem of Flight Delays important? 
It is seen that flight delays lead to a multitude of problems, including but not limited to, passenger inconvenience, re-scheduling issues for airlines' staff  and most importantly economic losses for both airlines and travellers. Currently, the average cost per flight is approximately [$47 per flight](https://www.airlines.org/dataset/u-s-passenger-carrier-delay-costs/). Moreover, air travellers are now spending billions of dollars on air travel yearly, with an **annual cost of only delay flights amounting to be $28 billion in 2018**, as estimated by the [Federal Aviation Administration](https://www.airlines.org/dataset/u-s-passenger-carrier-delay-costs/).

### Current State of the Art solution
The current state-of-the-art model that helps predict flight delays is a deeep Learning model which is well-suited for high-level complexity and massive data, and is called the
**Stacked Denoising Autoencoders and LM Algorithm**. It is based on the research done by [Maryam Farshchian Yazdi in 2020](https://link.springer.com/article/10.1186/s40537-020-00380-z), and focuses on using the Levenberg-Marquart Algorithm as the baseline.

### The Solution Approach

For this problem, our goal is to use feature elements from different datasets to predict whether or not flights will be delayed by 15 or more minutes. We believe this is important both for the airline companies and the travellers, as it will provide both user groups sufficient time to deal with issues around flight delays and be more prepared for the situation. Our goal is to be able to increase precision of predicting flight delays for flights that are actually delayed, and reducing our false positives

### Evaluation Metric

Given that the Exploratory data analysis (see below) shows that the outcome variable is heavily imbalanced toward the negative outcome (flights not delayed), and in light of our solution approach mentioned above, the appropriate evaluation metric of choice for algorithms is **area under precision-recall curve (auPRC)**. 

$$auPRC = \int_{0}^1 \frac{True Positives}{True Positives + False Positives} d (\frac{True Positives}{Positives})$$

This is a useful measure of success of prediction to handle the imbalance among target class. It helps answer two main questions about each algorithm's performance:
- How many positive cases have been classified correctly out of all positive cases in the data?
- How many true positives out of all that have been predicted as positives? 

A good model will have a high  value of auPRC which would represent high recall &  high precision. Here, high precision relates to a low number of positives marked as false (FPR), and high recall relates to a low number of negatives marked as false (FNR) .

Specifically, for the problem at hand, auPRC measures _how many delayed flights have been classified correctly out of all delayed flights in the data_, and focuses on the predictive sides as compared to other evaluation metrics.

The auPRC baseline for the data, that is the proportion of positives was found to be _0.188_

### About the Datasets

There were three datasets provided to help solve this problem of predicting flight delays, and their details are as follows:
1. _Airlines_:  This is a subset of the passenger flight's on-time performance data taken from the TranStats data collection available from the [U.S. Department of Transportation (DOT)](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time), for the time period of Jan 1, 2015 to Dec 31, 2019. This is the starting dataset for answering the problem at hand as it will help us find key details about the flights along. The dimensions for this data are 63,493,682 rows, and 109 columns

2. _Weather_: It is common knowledge that flight departures often get affected by weather conditions, which is why this data is important for us to look into for this problem. It will help us analyze the weather data corresponding to the origin and destination airports at the time of departure and arrival respectively and build features based upon this data. This data was downloaded from the [National Oceanic and Atmospheric Administration repository](https://www.google.com/url?q=https://www.ncdc.noaa.gov/orders/qclcd/&sa=D&source=editors&ust=1628289920180000&usg=AOvVaw29C4yxFG1OcA2jsAvEQnzc), for the time period of Jan 1, 2015 to Dec 31, 2019. There are 630,904,436 rows and 177 columns in this dataset.

3. _Stations_: This was another dataset that is related to the business problem, it contains information about weather stations and has 5,004,169 rows and 12 columns. However, we decided not to include this in our final dataset as it provided repetitive information that was already found in the other two datasets, for the context of this exercise.

#### Data Preparation
Before joining the Airlines dataset and the Weather dataset, we did some data preparation. We first checked for null values and decided that columns with greater than or equal to 95% null values wouldn't provide much useful information because a majority of them were string columns and looking at the NOAA data documentation, we saw that to impute them wouldn't be very useful. For the both datasets, we dropped columns with greater than or equal to 95% null values, which dropped 48 airline columns and 132 weather columns. For the airline dataset, we converted departure times from local time to UTC because we will need it to join with the weather data. Another thing we did for the airline data was map origin cities to a latitude and longitude. For the weather data, given the latitude and longitude, we mapped each row to the closest origin city from the list of cities in the airlines data.

<img src="https://github.com/AshwiniBhingare203/images261/blob/main/AirlineDataPrep.PNG?raw=true" width="10%">
<img src="https://github.com/AshwiniBhingare203/images261/blob/main/weatherdataprep.PNG?raw=true" width="10%">

## Joining the Datasets 
Please see **[join_notebook](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/join_notebook.py)** for details on the specific code


<img src="https://github.com/AshwiniBhingare203/images261/blob/main/datajoinstep1.PNG?raw=true" width="20%">
<img src="https://github.com/AshwiniBhingare203/images261/blob/main/datajoinstep2.PNG?raw=true" width="13%">

#### Joining Airline Data with Weather Data
We first joined the data on calendar day and the mapped city. Then we calculated the time difference between the weather data and flight using the timestamps of weather and flight. After, we filtered the earliest row per flight with a difference between 2 to 4 hours before the flight time. We chose a time difference window of 2-4 hours because we are predicting flight delay 2 hours before the scheduled departure time and weather data over 4 hours before departure would be stale. There were some rows that didnt fit in the 2-4 hours window, so we unioned back the 3 million rows to make sure we aren't losing data. Our fully joined dataset had 23,941,956 rows.

## Exploratory Data Analysis <a name="Section3"></a>

##### Delay Types
    We tried to do EDA on all the dealy types to understand what is mostly causing the departure delay. 
    As you can see in below EDA notebook 86% of flight depart on time..and 7.3% delays are because of late-aircrft delay and 2.3 % delays are due to weather . 
    Then we further investigated on weather data set to get insight on weather conditions at the type of flight departure
    
    We also  try to dig into cancellation to undersatnd the cause of cancelling the flight and our EDA shows that 80 percent of the time flights are cancelled due to weather condition . 
 ###### EDA nodebooks:  [html](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/EDA_Ashwini.html)  ,  [ipynb](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/EDA_Ashwini.html), [databricks](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/EDA_Ashwini.py)
  
  ###### EDA snapshots:
  <img src="https://github.com/AshwiniBhingare203/images261/blob/main/DelayTypesEDA.PNG?raw=true" width="15%%">
  <img src="https://github.com/AshwiniBhingare203/images261/blob/main/cancellationDelayEDA.PNG?raw=true" width="15%%">
  
  
   0 =NO_DELAY, 1 = WEATHER_DELAY, 2=SECURITY_DELAY, 3 = NAS_DELAY, 4 = LATE_AIRCRAFT_DELAY 
   
   C = carrier, B = weather, A = NAS
   
  ##### Frequent delays recorded by US states and airlines
      This plot shows the recorded delay in the differnt region on the US Map  - shows CA, Texas, florida  recorded the most number of flight details .
  
   ###### EDA notebooks:  [html](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/EDA_Ashwini.html)  , [ipynb](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/EDA_Ashwini.html), [databricks](https://github.com/UCB-w261/su21-team-team-   09/blob/main/FinalProject/EDA_Ashwini.py)    
      
   <img src="https://github.com/AshwiniBhingare203/images261/blob/main/delayOnMap.PNG?raw=true" width="30%%">

##### Correlation
We made correlation matrix to see which features are highly correlated with each other. We were interested in which features were most highly correlated with the target variable `DEP_DELAY` because we want to make sure we don't lose those features when dropping columns. We selected the the most highly correlated features with `DEP_DELAY` for our very first model before the mid-project presentation. 
  
<img src="https://github.com/AshwiniBhingare203/images261/blob/main/Summary%20Correlation%20Heatmap.png?raw=true" width="50%%">

We also used the correlation matrix to find features that are highly correlated with other features to take care of multicollinearity.

<img src="https://github.com/AshwiniBhingare203/images261/blob/main/HighCorrelation.png?raw=true" width="25%%">
###### EDA notebooks: [html](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/final_project_EDA_rochelle.html) and [databricks](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/final_project_EDA_rochelle.py)

##### Weather data
The weather data had multiple exploration gateways, so we selected the top columns that showed correlation and were supported by [research done on weather type flight delays, by Federal Aviation Administration](https://www.faa.gov/nextgen/programs/weather/faq/#faq3). From this, we selected that sky coverage, wind speed and atmospheric observation were the most relevant data for this problem. It is seen that the distribution for these three variables was skewed for the sky coverage and windspeed, but was normally distributed in the data for the atmospheric pressure observation.

  <img src="https://github.com/AshwiniBhingare203/images261/blob/main/weatherEDA.png?raw=true" width="50%%">
 ###### EDA notebooks: [html](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/final_project_EDA_simran.html) and [databricks](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/final_project_EDA_simran.py)

## Feature Engineering  & Pre-processing 

Please see **[final_model_feature_engineering](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/final_model_feature_engineering.py)** notebook

### Simple New Features
A significant amount of effort was put into feature engineering because it was necessary to improve model performance and model runtime. We started off simple minded and ran into a few walls. We started off by creating new features `HOUR` and `MINUTE` by extracting the hour and minute from `LOCAL_TIME`. We were interested in `HOUR` and `MINUTE` because we suspected a relationship between the time of day and the number of flights, amount of air traffic, and amount of activity in the airports, which can have an affect on actual departure times and flight delays. The next feature we created was `PRIOR_DAY_NUM_DELAY`, which represents the number of delayed flights that occurred the day before. We suspected that the number of delayed flights the day before has an affect on whether or not a flight is delayed because there could be residual effects from the previous day that bleeds into the next day.

### Weather Features
The next thing that we tackled were the weather features. At first glance, the weather features were very intimidating because there were a lot of them and it looked like each weather feature had multiple comma separated components. This is when we really delve deep into the [weather data documentation](https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf). We split the weather features on the commas into separate features for each component. Looking for ways to decrease the number features that resulted from hot encoding, we saw the opportunity for that in the ~90 newly created weather features. After the split, all ~90 weather features were string types, even the numerical data. 33 of the ~90 weather features were converted to DoubleType or IntegerType.

### Drop Columns
We first dropped columns with more than 95% missing values because they didn't provide enough information to justify keeping them. We also dropped features with data that wew would not have access to at the time of prediction. The next category of dropped features are features that provided duplicate or redundant information. We also looked at feature correlation with other features to control multicollinearity. Features highly correlated with another existing feature were dropped.

### Univariate Feature Selection
Univariate feature selection examines each feature individually to determine the strength of the relationship of the feature with the outcome [[source](https://blog.datadive.net/selecting-good-features-part-i-univariate-selection/)]. The selector was fit on the scaled columns. The strength metric used was the family-wise error rate, which is the probability of making at least one Type 1 error and selects features whose p-values are below a threshold that is scaled by 1 / numFeatures, thus controlling the family-wise error rate of selection. It helped reduce our columns from ~2800 to ~1100

### Pre-processing

To get the data ready for the the algorithm modeling stage, there were multiple steps needed to pre-process the data. By using the Spark pipeline functions, this pre-processing was scalable across the training and test dataset. For more details on the code, please refer to the **[final_project_model](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/final_project_model.py)** notebook. We first found the categorical and numerical columns in the data by the data type and then fed the train, test data along with columns names into a multi-step function. The steps taken for pre-processing were as follows:
The string columns are -- and the numeric columns are -- 

Next, we indexed the categorical columns (using the `StringIndexer()`). All string columns of categorical columns are mapped to a column of label indices. This is ordered by label frequencies so the most frequent label gets index 0. For this method, we did not drop the null values. To complete the processing for the  categorical columns, we one Hot encoded the categorical columns (using the `OneHotEncoder()`.The one hot encoder maps a categorical column with label indices to columns of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included because it makes the vector entries sum up to one, and hence linearly dependent. 

The following step of the pipeline was to impute the numerical column (using the `Imputer()`). Here in, we filled the missing values for each of the numeric columns with the median value of the column. 

Once the processing for the specific categorical and numeric columns was complete, the next step was to vectorize each of the transformed columns (using the `VectorAssembler()`). In this step, the transformed columns are merged together into a single vector column, excluding the target column. 
Additionally, the vectorized columns were scaled (using the `StandardScaler()`). Once the transformed columns are merged to one vector column, they are then scaled by centering the data with the mean and sccaled to unit standard deviation. The standardization of features is done by removing the mean and scaling to unit variance using column summary statistics on the samples in the training set. The unit standard deviation is computed using the corrected sample standard deviation, which is computed as the square root of the unbiased sample variance. Lastly, the original columns are dropped from the training and test set. Only the scaled and vectorized single column remains, along with the target column.

## Final Algorithm Theory

The model that performed the best was the Gradient Boosted Trees (GBT) Classifier. It is a emsemble learning classififcation model, which means a collection of weak learners are used to build the final additive model. Boosting means to modify subsequent trees to improve the model. Gradient Boosting is an algorithm that minimizes a loss function by iteratively choosing a function that points towards the negative gradient. The loss function estimates how good the model is at making predictions. We used the binomial deviance loss function, which is the negative binomial log-likelihood loss function for binary classification.

The first thing the algorithm does is make initial predictions by calculating the log of the odds of the target feature. It then converts that value into a probability by using a logistic function. For each row in the the training data, the residual (observed value minus the predicted value) is calculated. Then a new decision tree is built and that tree tries to predict the residuals that were calculated in the step before. Instance values are transformed using the formula:
$$\frac{\sum{\text{Residual}}}{\sum{[\text{PreviousProb}*(1-\text{PreviousProb})]}}$$

To make new predictions, the log of the odds prediction is calculated for each instance on the training data and then converted into a probability. The formula for predictions is:
$$ \text{baseLogOdds} + \text{learningRate} * \text{predictedResidualValue}$$

The learning rate hyperparameter is used to controll how much the previous tree contributes to the new tree. The process of calculating new residuals and creating new trees to fit new residuals is repeated until the threshold is met.

Sources:
- [Gradient Boosting In Classification: Not a Black Box Anymore!](https://blog.paperspace.com/gradient-boosting-for-classification/)
- [Machine Learning Algorithms: A comparison of different algorithms and when to use them](https://www.linkedin.com/pulse/machine-learning-algorithms-comparison-different-when-taniya-ghosh/)
- [Gradient Boosting Classification explained through Python](https://towardsdatascience.com/gradient-boosting-classification-explained-through-python-60cc980eeb3d)

### Cross validation 
  We chose Blocked Cross validation for validating our models performance . Block Cross validation works by adding margins at two positions . The first is between the training and validation folds in order to prevent the model from observing lag values which are used twice, once as a regressor and another as a response. The second is between the folds used at each iteration in order to prevent the model from memorizing patterns from an iteration to the next. 
  
  In Blocked Cross validation , data are split strategically rather than randomly .  Each fold is generated by excluding nearby observations around each testing point within the specified distance. This  method prevent from introducing leakage from future data to the model. Since clocked cross validation adds margin at two positions , it prevents model from observing future pattern and forecast or try to memorize from previous time series observation .
  
   To implement this we orderned our vectorized data by time and slpit it into 3 parts (###### 3-fold cross validation) and then futher each part we split into 2 parts for 80% train and 2o% test.
   
   <img src="https://github.com/AshwiniBhingare203/images261/blob/main/CrossValidation.PNG?raw=true" width="20%%">

## Algorithm Exploration <a name="Section2"></a>

For our baseline, we tried multiple algorithms on our training set and our expectation was that with each of them the evaluation metric, the auPRC improves. Here is what we tried:
Firstly, we explored **Logistic Regression** model because it is one of the simplest & fastest models, performs well on data sets with linear separation boundaries, which can be found in our dataset. We fit this model only on the training set's numerical columns, and noticed that without any form of cross validation, the auPRC was _0.394_. This proved as an improvement as compared the baseline of auPRC observed in the data previously (i.e. 0.188)

Next, we tried the **Support Vector Machine** (SVM) model, as it has the ability to solve linear and non-linear problems and work well for many practical problems. For this specific exploration, we focused on using the _linear_ version of the support vector machine model as it has shown to scale better with Spark. The expectation was that using these support vectors, we maximize the margin of the classifier and help increase the auPRC. The auPRC observed on the full training data was _0.286_ after block cross validation, with the best auPRC score achieved at regularization parameter of 0 

In the spirit of exploration, the next model on our list was **Gradient Boosted Trees Classifier**(GBT). This ensemble learning classification model has exhibited the capability to capture extremely complicated non-linear patterns, which was seen in our exploratory data analysis. It supports binary labels, which is the target column as wel as both continuous and categorical features. When we fit this model on the training set, the auPRC was _0.456_ after block cross validation, and the best auPRC score was achieved at 10 tree depth.

Lastly, the next model we tried was the **Gaussian Naive Bayes** model. We used this model as it was the easiest to understand when described using binary or categorical input values, which made a good proportion of our input features. The trade-off for this model is that it makes the naïve assumptions that features are conditionally independent and extending 
it to real-valued attributes, by assuming a Gaussian distribution. When we fold this model on the training data, after block cross validation, the auPRC was found to be _0.206_ with the best smoothing at factor of 1.5. 

For detailed code, please refer to the [**final project model**](https://github.com/UCB-w261/su21-team-team-09/blob/main/FinalProject/final_project_model.py)

## Conclusion<a name="Section5"></a>

| Metric      | Logistic Regression | SVM           | GBT           | Gaussian NB |
| :---        |    :----:           |          ---: |          ---: |        ---: |
| auPRC      | 0.394               | 0.286         | **0.456**     | 0.206       |
| Accuracy    | 80.4%               | 80.4%         | **82.2%**     | 62.6%       |
| Time        | 0.05hr              | 4.93hr        | **1.92hr**    | 1.06hr      |

The above table comparison shows that the Gradient Boost Tree classifier has the best performance, in comparison to the other models, for both auPRC and overall accuracy. In terms of scaling it, it did show that the model took approximately ~2hrs to fit on the full training set of data which was less than the SVM, which took the maximum time. While this was not optimal, the GBT was the most valuable model in terms of efficiency for evaluation and time performance. _The auPRC for GBT was 1.42x times higher than that found in the original training data_. The confusion matrix of the GBT were as follows:

| True/Predicted      | 0 | 1  |
| :---|    :----:|          ---: |  
| 0 | 79.6% | 0.8% | 
| 1 | 17% | 2.6% | 


As we can see in the confusion matric, the % of flights that were not delayed and predicted as not delayed is the highest (TN) with the the flights that were delayed being predicted as not delayed (FN) as the second highest. The predictions from the GBT model shows that the problem of predicting flight delays was solved to an extent as we were able to maximize the precision of the predicted delayed flights, even if it was a minor percentage of the full dataset. With confidence, we can predict flights that were not delayed, but predicting flights that were delayed comes with some cautionary steps around the modeling.

## Application of Course Concepts<a name="Section6"></a>

<!---
- Spark: RDDs, Monitoring and Debugging, Performance Tuning, Parallel Computing
- PCA
- Decision Trees
- Data Systems and Pipelines
--->

Spark was a big component in this course, and in the assignment, we took advantage of the open source parallel computation framework. We used spark's big data pipeline to process and query the large amount for flight data given to us. Initially, with just the first four months of data, we got away with Python Pandas dataframes to do EDA. However, after the mid-project presentations, with the full dataset, we realized that Python Pandas won't be enough for our tasks, even though it was a tool that we were all familiar with. We were able to perform our EDA tasks with the Spark framework. In the asynchronous material, memory usage comes up throughout the course. We learned that memory over the has gotten cheaper and more accessible. We learned and were reminded about how important caching is and would be in this assignment. We used **cache()** statements because working from memory is more efficient and to avoid redundant shuffles. We also learned about the directed acyclic graph (DAG), which represents the execution plan for the driver code. Looking into the DAG proved to be very useful from the start when we were working on the join of the airplane and weather data. One of the initial join jobs that we attempted had a very long and complex DAG, but we were able to streamline the task. We also learned the importance of the Spark UI, which we used to monitor and debug our jobs.

We decided to use the Gradient Boosted Tree (GBT) Classifier as our final model because we learned in the asynchronous material that decision trees are very popular in machine learning, are highly scalable and parallelizable, and tend to have high performance. We opted for the GBT classifier, a ensemble learning classification model, because we learned that to make decision trees more useful, we can learn ensembles of decision trees. To improve the quality of predictions, we can learn a collection or ensemble of decision trees. In decision tree boosting, each decision tree depends on the previous decision tree's predictions as a target variable to learn over. Each subsequent tree is reweighted to focus on errors. Another reason for choosing the GBT model is because we had a long battle with feature selection and feature engineering. We learned that feature/variable selection is part of and included in decision tree learning.

As expected, we spent a lot of energy on figuring out what can we do to preprocess our data to minimize runtime and resources used in modeling. We learned that GBT does not require a lot of preprocessing on data to work, but preprocessing can improve performance and runtimes. We also learned that other models like SVM and OLS require more preprocessing, so we thought it would probably be beneficial to work on preprocessing. We had a similar thought process for feature selection. We knew that GBT had built in feature selection, but other algorithms that we wanted to use did not come with it. And we suspected that some dirty work in feature selection before modeling will still improve the GBT classifier.